In [5]:
from PCSampler import PCSampler, vis_pc, transform_cloud
from manifpy import SE3, SO3, SE3Tangent
import numpy as np

In [2]:
S = PCSampler()
S.load_samples()
clouds = S.get_clouds()
transforms = S.get_transforms(use_quat=True)

In [45]:
#get the two first clouds and their tranformations to world
c0, c1 = clouds[7], clouds[8]
T0, T1 = SE3(transforms[0]), SE3(transforms[1])
c0_world = transform_cloud(c0, T0.transform())
c1_world = transform_cloud(c1, T1.transform())
vis_pc([c0_world, c1_world])

In [46]:
# get ground truth relative transformation
T = (T1 - T0).exp()
c1_0 = transform_cloud(c1, T.transform())
vis_pc([c1, c1_0])


In [ ]:
# TODO give c1 some random perturbations around T 

In [59]:

#registering c1 to c0
import open3d as o3d

#convert clouds to open3d clouds
pcd0 = o3d.geometry.PointCloud()
pcd0.points = o3d.utility.Vector3dVector(c0)
pcd0.estimate_normals()

pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(c1)
pcd0.estimate_normals()

#do registration
treshold = 1
initial_T = np.eye(4)
reg = o3d.pipelines.registration.registration_icp(
    pcd1, pcd0, treshold, initial_T, 
    o3d.pipelines.registration.TransformationEstimationPointToPlane()
)
icp_transform = reg.transformation
print(icp_transform)
c1_registered = transform_cloud(c1, icp_transform)

#vis_pc([c0, c1_0])
vis_pc([c0, c1_registered])

[[ 9.91238116e-01  1.32057797e-01  2.78119815e-03  1.77350588e+00]
 [-1.32052985e-01  9.91240935e-01 -1.84875799e-03 -1.62837071e-01]
 [-3.00098036e-03  1.46529387e-03  9.99994423e-01  1.95985847e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
